In [ ]:
# Query 
from __future__ import division
import pyodbc, sys, socket
# Web Scrappnig
import requests
from bs4 import BeautifulSoup
# RegEx / Regular Expressions
import re
from re import sub
#  Json
import json

In [ ]:
arg=socket.gethostname()
cnxn = pyodbc.connect('DRIVER={DBMAKER};SERVER=' + arg + ';DATABASE=COTRIPALDB_SIMPLUS_BASE;UID=SYSADM;PWD=')
cursor = cnxn.cursor()

sql=("select a.gtin , a.codigo_cotripal, cast(a.descricao as varchar(250)) as nome, a.id_erp_departamento, a.id_erp_setor, a.id_erp_item from produto as a where a.id_erp_departamento = 018 and a.descricao like '%VV%' ORDER BY IDX;")


cursor.execute(sql)

rows = cursor.fetchall()

cursor.close()

In [ ]:
headers = {
    "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"
}
global url
url = 'https://www.vv.com.br/pt/produto/detalhe/'
global infos
infos = []

In [ ]:
def findVVCod(nome):
    indice = nome.find('VV')
    nome = nome[indice:]
    nome = nome.replace(' ', '/')
    nome = nome.split('/')
    cods = []
    for cod in nome:
        if 'VV-' in cod:
            cod = cod.replace('VV-', '')
            cod = re.sub('[A-Z]', '', cod)
            cods.append(cod)
    return cods

In [ ]:
def search(row):
    cod = str(findVVCod(row[2])[0])
    if not '-' in cod:
        print(cod)
        print('não foi')
        return
    link = str(url + cod)
    req = requests.get(link, headers=headers)
    if req.status_code != 200:
        print(req.status_code)
        return
    site = BeautifulSoup(req.text, "html.parser")
    equipamentos = site.find("ul", class_="listarEquipamentos")
    if not equipamentos:
        print('Não encontrado')
        return
    equipamentosList = []
    for li in equipamentos.contents:
        if len(li) > 1:
            keys = []
            values = []
            for el in li.find_all("h3"):
                keys.append(el.get_text())
            for el in li.find_all("p"):
                values.append(el.get_text())
            try:
                equipamentosList.append({ keys[0] : values[0], keys[1] : values[1], keys[2] : values[2:] })
            except:
                equipamentosList.append({ keys[0] : values[0], keys[1] : values[1] })
    urlImagens = []
    imagens = site.find("ul", class_="listaGaleria")
    imagens = imagens.find_all("img")
    for img in imagens:
        urlImagens.append("https://www.vv.com.br/" + img["src"])
    nome = site.find("div", class_="subtitulo")
    nome = nome.find("h2").get_text()
    descricao = site.find("div", class_="descricao")
    descricao = descricao.find("ul")
    descricao_keys = []
    descricao_values =[]
    for li in descricao.contents:
        if len(li) > 1:
            for el in li.find_all("h3"):
                descricao_keys.append(el.get_text())
            for el in li.find_all("p"):
                descricao_values.append(el.get_text())

    itemInfo = {
        "gtin" : row[0],
        "codigo_cotripal" : row[1],
        "id_erp_departamento" : row[3],
        "id_erp_setor" : row[4],
        "id_erp_item" : row[5],
        "nome" : nome,
        "imagens" : urlImagens
    }
    for i, key in enumerate(descricao_keys):
        itemInfo.update({key : descricao_values[i]})
    itemInfo.update({"Equipamentos":equipamentosList})
    infos.append(itemInfo)

In [ ]:
for row in rows:
    search(row)

In [ ]:
print(json.dumps(infos, ensure_ascii=False))